# データ確認


- 調布市以外のデータを確認
    - データ数を確認
    - PoIの数、種類、分布を確認

In [3]:
import pandas as pd
import os
import json
import csv
import glob
import collections
import matplotlib.pyplot as plt
import numpy as np

In [4]:
print(os.getcwd())  # 例: '/Users/username/projects'

/Users/osamu/study/poi_sim/src


### 0. データの読み込み

In [40]:
# Load

#軌跡データ　ファイル一覧取得
trace_path = "/Users/osamu/study/poi_sim/data/exe_data/"
files = sorted(os.listdir(trace_path))  
# display(files)



# データ内容確認　（all, defalt, home , output)
trace_path = "/Users/osamu/study/poi_sim/data/origin/trace/20190201.csv"
default_path = "/Users/osamu/study/poi_sim/data/exe_data/201902_week1.csv"
home_path    = "/Users/osamu/study/poi_sim/data/exe_data/201902_week1_home.csv"
output_path  = "/Users/osamu/study/poi_sim/data/exe_data/201902_week1_output.csv"

trace_df = pd.read_csv(trace_path, index_col=0)
default_df = pd.read_csv(default_path, index_col=0)
home_df    = pd.read_csv(home_path, index_col=0)
output_df  = pd.read_csv(output_path, index_col=0)

def drop_unnamed_columns(df):
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]

trace_df   = drop_unnamed_columns(trace_df)
default_df = drop_unnamed_columns(default_df)
home_df    = drop_unnamed_columns(home_df)
output_df  = drop_unnamed_columns(output_df)

display(trace_df.head(20))
# display(default_df.head(5))
# display(home_df.head(5))
# display(output_df.head(5))


# print("trace_df",len(trace_df))
# print("default_df",len(default_df))
# print("home_df",len(home_df))
# print("output_df",len(output_df))


In [9]:
print("trace_df",trace_df["citycode"].value_counts())
print("default_df",default_df["citycode"].value_counts())
print("home_df",home_df["citycode"].value_counts())
print("output_df",output_df["citycode"].value_counts())

trace_df 13208.0    1885056
13206.0     629105
13112.0     572211
13104.0     488290
13113.0     371759
            ...   
33103.0          1
23236.0          1
17209.0          1
7545.0           1
16342.0          1
Name: citycode, Length: 788, dtype: int64
default_df 13208.0    760419
13206.0    254200
13112.0    220552
13104.0    183198
13113.0    147769
            ...  
8447.0          1
6341.0          1
6208.0          1
4324.0          1
20481.0         1
Name: citycode, Length: 1156, dtype: int64
home_df 13208.0    584034
13206.0    190424
13112.0    138533
13104.0    113505
13201.0    111497
            ...  
2361.0          1
47315.0         1
46534.0         1
12342.0         1
27108.0         1
Name: citycode, Length: 1039, dtype: int64
output_df 13208.0    568910
13206.0    184663
13112.0    134082
13104.0    109420
13201.0    108009
            ...  
40384.0         1
17210.0         1
40210.0         1
1230.0          1
6211.0          1
Name: citycode, Length: 1020, d

### 1. 隣接市町村　
＊６桁目はチェック数字
- 131121	世田谷区	せたがやく
- 132047	三鷹市	みたかし
- 132101	小金井市	こがねいし
- 132063	府中市	ふちゅうし
- 132195	狛江市	こまえし
- 132250	稲城市	いなぎし

#### 1.1 各市区のデータ数確認

In [15]:
city_map = {
    13112.0: "世田谷区",
    13204.0: "三鷹市",
    13210.0: "小金井市",
    13206.0: "府中市",
    13219.0: "狛江市",
    13225.0: "稲城市"
}

counts = trace_df['citycode'].value_counts()
result = counts[counts.index.isin(city_map.keys())]

result_df = result.reset_index()
result_df.columns = ['citycode', 'count']
result_df['city'] = result_df['citycode'].map(city_map)
result_df = result_df[['citycode', 'city', 'count']]

print(result_df)

   citycode  city   count
0   13206.0   府中市  629105
1   13112.0  世田谷区  572211
2   13204.0   三鷹市  334908
3   13225.0   稲城市  155185
4   13219.0   狛江市  124623
5   13210.0  小金井市   49506


#### 1.2 PoI大カテゴリのユニークを確認

In [32]:
import re

# 文字列から先頭の要素（大分類）を抜き出す関数
def fast_get_major(x):
    if isinstance(x, str) and x.startswith("["):
        # 先頭のシングルクオートで囲まれた文字列を正規表現で取る
        m = re.match(r"\['([^']+)'", x)
        return m.group(1) if m else None
    else:
        return x

# 大分類を抽出
trace_df['poi_major'] = trace_df['poi'].map(fast_get_major)

# ユニーク一覧
unique_major = trace_df['poi_major'].unique()
print(sorted(unique_major))


['NA', 'OoR', 'move', 'その他サービス', 'スポーツ・趣味娯楽・レジャー', '不動産・賃貸・展示場', '交通・運輸・倉庫', '出版・印刷', '医療・薬・保険衛生', '学校・趣味教室・図書館', '官公庁・団体・福祉', '建設・工事', '旅行・観光・温泉・旅館・ホテル', '自動車・オートバイ・自転車・ドライブ', '製造・加工', '販売・卸', '農林・水産・鉱業', '金融・保険・証券', '電気・ガス・通信・放送・新聞', '飲食店']


#### 1.3 隣接する各市区のPoI頻出上位１０件取得

In [35]:
import ast

def get_major_category(x):
    try:
        if isinstance(x, str) and x.startswith("[") and x.endswith("]"):
            parsed = ast.literal_eval(x)
            return parsed[0] if parsed else None
        else:
            return x
    except:
        return x

trace_df['poi_major'] = trace_df['poi'].apply(get_major_category)

city_map = {
    13112.0: "世田谷区",
    13204.0: "三鷹市",
    13210.0: "小金井市",
    13206.0: "府中市",
    13219.0: "狛江市",
    13225.0: "稲城市"
}
trace_df['city'] = trace_df['citycode'].map(city_map)

city_major_counts = (
    trace_df.groupby(['city','poi_major'])
    .size()
    .reset_index(name='count')
)

top10_each_city = (
    city_major_counts
    .sort_values(['city','count'], ascending=[True,False])  # 都市ごとに count 降順
    .groupby('city')
    .head(10)  
)

print(top10_each_city)


     city           poi_major   count
0     三鷹市                move  158872
1     三鷹市             その他サービス   24976
7     三鷹市         学校・趣味教室・図書館   24106
12    三鷹市                販売・卸   23373
9     三鷹市               建設・工事   19591
11    三鷹市               製造・加工   17200
10    三鷹市  自動車・オートバイ・自転車・ドライブ   13984
8     三鷹市           官公庁・団体・福祉   13224
6     三鷹市           医療・薬・保険衛生   10847
4     三鷹市            交通・運輸・倉庫    8551
18   世田谷区                move  342551
29   世田谷区  自動車・オートバイ・自転車・ドライブ   32064
19   世田谷区             その他サービス   31145
31   世田谷区                販売・卸   28655
24   世田谷区           医療・薬・保険衛生   25230
27   世田谷区               建設・工事   21248
25   世田谷区         学校・趣味教室・図書館   20972
26   世田谷区           官公庁・団体・福祉   19718
34   世田谷区                 飲食店   17159
30   世田谷区               製造・加工   10357
35   小金井市                move   25335
42   小金井市         学校・趣味教室・図書館    4529
47   小金井市                販売・卸    3385
36   小金井市             その他サービス    2770
46   小金井市               製造・加工    2688
43   小金井市   

#### 1.4 小分類までみる

In [18]:
trace_df['city'] = trace_df['citycode'].map(city_map)

top_pois = (
    trace_df.groupby('city')['poi']
    .apply(lambda x: x.value_counts().head(10))
    .reset_index()
)

top_pois.columns = ['city', 'poi', 'count']

print(top_pois)

    city                                                poi   count
0    三鷹市                                               move  158872
1    三鷹市  ['自動車・オートバイ・自転車・ドライブ', 'パーキング・駐輪場', '駐車場業', '自...    5475
2    三鷹市  ['医療・薬・保険衛生', '薬局・薬店', '薬局・調剤薬局', '医療・薬・保険衛生',...    5162
3    三鷹市  ['学校・趣味教室・図書館', '幼稚園・保育園', '保育園・保育所', '学校・趣味教室...    4359
4    三鷹市  ['学校・趣味教室・図書館', '語学スクール', '幼児英語教室', '学校・趣味教室・図...    3949
5    三鷹市  ['官公庁・団体・福祉', '公共施設関連', '公共施設', '官公庁・団体・福祉', '...    3699
6    三鷹市  ['学校・趣味教室・図書館', '学校・教室関連', '学童保育所', '学校・趣味教室・図...    3676
7    三鷹市  ['製造・加工', '木製品・紙・パルプ製品', '紙器製造業', '製造・加工', '木製...    3356
8    三鷹市  ['不動産・賃貸・展示場', '不動産取引', '不動産取引業', '不動産・賃貸・展示場'...    3303
9    三鷹市  ['建設・工事', '建設・工事業', '建設業', '建設・工事', '建設・工事業', ...    3161
10  世田谷区                                               move  342551
11  世田谷区  ['自動車・オートバイ・自転車・ドライブ', 'パーキング・駐輪場', '駐車場業', '自...   12484
12  世田谷区  ['医療・薬・保険衛生', '医療機関', '歯科', '医療・薬・保険衛生', '医療機関...    7937
13  世田谷区  ['自動車・オートバイ・自転車・ドライブ', 'レンタカー', 'レンタカー

#### 1.5 homeが調布市である隣接各市区のデータ数

In [39]:
trace_df[trace_df['citycode'] == 13208]

In [37]:
trace_df['home_citycode'].unique()

array([12240.,  6210., 13210., 13220., 13110., 11210., 22100., 14136.,
       13120., 13224., 13200., 12230., 13100., 14150., 20210., 12200.,
       11240., 11224., 12216., 20380., 12220., 19400., 14210.,  1101.,
       11330.,    nan, 11230., 14120., 11100., 14110., 12104., 11200.,
       12210., 13230., 14104., 34100., 11110., 22220., 19200.,  8220.,
       19220., 11220.,  7504.,  4104., 10530.,  7204., 28210., 19420.,
       26200.,  8200., 31200., 14130.,  8216., 25220.,  7210., 23100.,
       14216., 15200., 13304., 15210.,  9200., 11464., 24210., 10210.,
        1108., 27220., 27120.,  4100., 16200., 46200.,  8210.,  1105.,
       22200., 13360., 14200.,  3200., 33200.,  9340.,  9220.,  5200.,
       21220., 30200.,  5204., 27230., 11384., 29220., 23500., 20220.,
        8230., 44200., 33100.,     0., 13384., 28100., 28220.,  3216.,
       27100., 40100., 23120., 43100., 20420., 11304., 16210.,  9210.,
       10200.,  8440., 43200., 23230.,  8310., 30400.,  4610.,  1229.,
      

In [36]:
chofu_code = 13208.0  

chofu_home = trace_df[trace_df['home_citycode'] == chofu_code]

# 各 citycode ごとの件数を集計
chofu_counts = (
    chofu_home.groupby('citycode')
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
)

# 市町村名を付けたい場合
chofu_counts['city'] = chofu_counts['citycode'].map(city_map)

print(chofu_counts)


Empty DataFrame
Columns: [citycode, count, city]
Index: []
